In [11]:
from bs4 import BeautifulSoup
import urllib.request
import re
import os

### Download midi data

In [21]:
def getMidiURLs(searchTokens):
    result = []
    url_root = 'http://www.mazurka.org.uk/ana/midi'
    r = urllib.request.urlopen(url_root).read()
    soup = BeautifulSoup(r, "lxml")
    anchors = soup.find_all('a', class_='colon')
    print(anchors)
    for searchToken in searchTokens:
        listURLs = []
        for anchor in anchors:
            if re.search(searchToken, anchor.get('href')):
                trows = anchor.parent.parent.contents[4].table.contents
                for trow in trows:
                    if trow != '\n' and trow.contents[2].a:
                        href = trow.contents[2].a.get('href')
                        tokenid = href.split('/')[0]
                        url = '%s/%s/%s-avgtapsd0.mid' % (url_root, tokenid, tokenid)
                        listURLs.append(url)
                break
        result.append(listURLs)
    return result

In [22]:
def downloadData(urls, download_dirs, root_dir):
#     os.mkdir(root_dir)
    for i in range(len(urls)):
        localdir = root_dir + '/' + download_dirs[i]
        os.mkdir(localdir)
        for url in urls[i]:
            localFile = '%s/%s' % (localdir, os.path.basename(url))
            urllib.request.urlretrieve(url, localFile)

In [23]:
midi_root = 'midi'
download_dirs = ['op30no2', 'op63no3', 'op68no3']
searchTokens = ['mazurka30-2','mazurka63-3','mazurka68-3']

In [24]:
midiURLs = getMidiURLs(searchTokens)

[<a class="colon" href="http://kern.ccarh.org/cgi-bin/ksdata?l=users/craig/classical/chopin/mazurka&amp;format=midi&amp;file=mazurka06-1.krn">Mazurka in F<img border="0" src="http://mazurka.org.uk/images/sharp.png"/> minor</a>, <a class="colon" href="http://kern.ccarh.org/cgi-bin/ksdata?l=users/craig/classical/chopin/mazurka&amp;format=pdf&amp;file=mazurka06-1.krn">Op. 6, No. 1</a>, <a class="colon" href="http://webzu.sapp.org/cgi-bin/webzualbum?user=craig&amp;action=download&amp;id=pid9058&amp;track=01">:</a>, <a class="colon" href="http://webzu.sapp.org/cgi-bin/webzualbum?user=craig&amp;action=download&amp;id=pid9166&amp;track=03&amp;disk=3">:</a>, <a class="colon" href="http://webzu.sapp.org/cgi-bin/webzualbum?user=craig&amp;action=download&amp;id=pid9174&amp;track=01">:</a>, <a class="colon" href="http://webzu.sapp.org/cgi-bin/webzualbum?user=craig&amp;action=download&amp;id=pid9062&amp;track=01">:</a>, <a class="colon" href="http://webzu.sapp.org/cgi-bin/webzualbum?user=craig&amp;

In [20]:
downloadData(midiURLs, download_dirs, midi_root)

### Download annotation data

In [47]:
def getAnnotURLs(midiURLs):
    url_root = 'http://www.mazurka.org.uk/ana/markup'
    result = []
    for ls in midiURLs:
        toAdd = []
        for midiURL in ls:
            tokenid = midiURL.split('/')[-2]
            annotURL = '%s/%s/%s-avgtap-sv.txt' % (url_root, tokenid, tokenid)
            toAdd.append(annotURL)
        result.append(toAdd)
    return result

In [48]:
annot_root = 'annot_audio'

In [49]:
annotURLs = getAnnotURLs(midiURLs)

In [50]:
downloadData(annotURLs, download_dirs, annot_root)